![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidgets_4 Gauges.

Let's use the previous work and connect the Phidget temperature and humidity sensors to a live gauge in Plotly.

We use Plotly to get some nice looking gauges, and make them live by attaching them to the Phidget data. 


## Set up gauges

We first set up the two gauges, for temperature and humidity

In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

ind_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

ind_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [5, 35]}}
)


fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}]])
fig.append_trace(ind_temp, row=1, col=1)
fig.append_trace(ind_hum, row=1, col=2)

f = go.FigureWidget(fig)

f

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0, 1]},
              'gauge': {'axis': {'rang…

## Setup the Phidget devices

You must be running on Chrome on your laptop or desktop, and the Phidget device needs to be connected to the USB port on your computer. 

As in the earlier notebooks, the main steps are thus:
1. Set up default values for variables that cross js cells
2. Define and open the USB connection
3. Open the Humidity and Temperature device
4. Request a connection to the Phidgets device
5. Read values from the devices
6. When we are done, close the device


One chenge though is in Step 3, where we add some come that will communicate with Python. It looks like this:
```
    IPython.notebook.kernel.execute("f.data[1]['value'] = " + humidity );  
and
    IPython.notebook.kernel.execute("f.data[0]['value'] = " + temperature );  

```

These lines communicate with our Python code and update the values in data structure **f.data**.

Step 5 then becomes "live" automatically. 

So run the following Python and Javascript code, and connect your Phidget.

## Step 0
Let's import a few Python libraries that we need. 

In [2]:
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

## Step 1.

In [3]:
%%js
window.usbconn = null;
window.humSensor = null;
window.tempSensor = null;

<IPython.core.display.Javascript object>

## Step 2. 

In [4]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});

require(['phidget22'], function(phidget22) {
    if (window.usbconn == null){
        window.usbconn = new phidget22.USBConnection();
        element.text("Creating a new USB Connection.");
    }
    console.log(window.usbconn)
    
    window.usbconn.connect().catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
    element.append(" Connected.");
});

<IPython.core.display.Javascript object>

In [5]:
## We rest for a bit while the devices connect
sleep(2)

### Confirm the connection 

In [6]:
%%js
element.text(window.usbconn.connected);

<IPython.core.display.Javascript object>

## Step 3

In [7]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});

require(['phidget22'], function(phidget22) {
        // set up the humidity sensor
    window.humSensor = new phidget22.HumiditySensor();
    window.humSensor.onHumidityChange = function (humidity) {
            IPython.notebook.kernel.execute(
                "f.data[1]['value'] = " + humidity
            );  
        };
    window.humSensor.open();
        // set up the temperatre sesor
    window.tempSensor = new phidget22.TemperatureSensor();
    window.tempSensor.onTemperatureChange = function (temperature) {
            IPython.notebook.kernel.execute(
                "f.data[0]['value'] = " + temperature
            );  
        };
    window.tempSensor.open();
});

<IPython.core.display.Javascript object>

In [8]:
## We rest for a bit while the devices connect.
sleep(2)

## Step 4

In [9]:
%%js
window.usbconn.requestWebUSBDeviceAccess();

<IPython.core.display.Javascript object>

## Step 5. PAUSE HERE

The gauges should be running now. Everything is live.  Scroll up and check it out the gauges, to see the temperature and humitity values. Try breathing on the sensor, so you can see changes in the readings. 

We can even display a copy of the gauges right here, also live. 

In [10]:
f

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0, 1]},
              'gauge': {'axis': {'rang…

## Step 6. Closing down the sensors

It is **really importnat** to close the sensor now, as otherwise they will keep busy forever, always trying to update the sliders with the latest values. So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [11]:
def doDisconnect(b):
    display(Javascript("""
    window.humSensor.close();
    window.tempSensor.close();
    setTimeout(() => { window.usbconn.close(); },1000);
    setTimeout(() => { window.usbconn.delete(); },2000);
    """))
    print("You have disconnected the Phidgets.")

run_button = widgets.Button(
      description = 'Click to disconnect'
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

Press this button when you are done, to disconnect the Phidgets


Button(description='Click to disconnect', style=ButtonStyle())

<IPython.core.display.Javascript object>

You have disconnected the Phidgets.


### Confirm

You can confirm the phidget is open or closed by running the following cell. 

If it is open (true), try clicking the button above, again.

In [12]:
%%js
element.text("Is the humidity sensor open? " + window.humSensor.isOpen);
element.append("<p>Is the humidity sensor open? " + window.tempSensor.isOpen);

<IPython.core.display.Javascript object>

## Conclusion

We have shown how to display the temperature and humidity values from the Phidget device using a live widget from the ipywidgets library. 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)